In [225]:
import pandas as pd
import numpy as np
import zipfile

In [491]:
with zipfile.ZipFile("../data/reddit_scrape.csv.zip") as z:
    with z.open("reddit_scrape.csv") as f:
        reddit_df = pd.read_csv(f)

reddit_df['Date'] = pd.to_datetime(reddit_df['Date'])    
reddit_df.head()

,Season,Week,Day,Date,Time (ET),Unnamed: 5,Favorite,Score,Spread,Unnamed: 9,...,Notes,GameHub/RedZone,Game thread link?,Post-game thread link?,Game thread,Post game thread,GameHub Scraped,Game thread scraped,Post game thread Scraped,Subscribers (in thousands)
0,2017,1,Thu,2017-09-07,8:30,@,New England Patriots,L 27-42,L -8,NaN,...,NaN,NaN,NaN,NaN,https://www.reddit.com/r/nfl/comments/6yr619/g...,https://www.reddit.com/r/nfl/comments/6ysc10/p...,NaN,"[3193, ('Dear Falcons fans: are you guys ready...","[2781, ('I\'m imagining a world where the Patr...",640
1,2017,1,Sun,2017-09-10,1:00,@,Buffalo Bills,W 21-12,W -7,NaN,...,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97il/g...,https://www.reddit.com/r/nfl/comments/6zajs1/p...,"[240, ('Bitter-sweet start to the NFL season w...","[109, ('I\'m a Jets fan. I\'m a Texas A&M fan....","[120, ('Bills number 1 seed in the AFCE!', 194...",640
2,2017,1,Sun,2017-09-10,1:00,NaN,Atlanta Falcons,W 23-17,L -6.5,@,...,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97gn/g...,https://www.reddit.com/r/nfl/comments/6zajuc/p...,"[240, ('Bitter-sweet start to the NFL season w...","[321, ('Wow this national anthem singer is a f...","[351, (""We just went down to the wire against ...",640
3,2017,1,Sun,2017-09-10,1:00,@,Cincinnati Bengals,L 0-20,L -2.5,NaN,...,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97hl/g...,https://www.reddit.com/r/nfl/comments/6zajr0/p...,"[240, ('Bitter-sweet start to the NFL season w...","[192, (""Man I hope Woodhead stays healthy for ...","[293, ('[Summary of this game](https://pbs.twi...",640
4,2017,1,Sun,2017-09-10,1:00,NaN,Pittsburgh Steelers,W 21-18,L -10,@,...,NaN,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97l9/g...,https://www.reddit.com/r/nfl/comments/6zajwi/p...,"[240, ('Bitter-sweet start to the NFL season w...","[367, (""1 drive no turnovers. WE'VE TURNED THE...","[383, (""Don't let this distract you from the f...",640


In [515]:
scrape_cols = [x for x in reddit_df.columns if 'craped' in x]
for col in scrape_cols: # converts strings to lists
    reddit_df[col] = reddit_df[col].apply(lambda x: eval(x) if type(x)==str else x)

reddit_df['all comments'] = [lst[1:] for lst in reddit_df.loc[:,'Game thread scraped']] \
                            + [lst[1:] for lst in reddit_df.loc[:,'Post game thread Scraped'] if type(lst)==list]
reddit_df.loc[reddit_df['all comments'].isna(),'all comments'] = \
        reddit_df.loc[reddit_df['all comments'].isna(),'Game thread scraped']


ValueError: Length of values (3130) does not match length of index (1567)

In [485]:
text = reddit_df['Game thread scraped'][0][1][0]
text

'Dear Falcons fans: are you guys ready for 17 straight weeks of hearing how you fucked up the Super Bowl? I feel your pain. \n\nSincerely, \n\n\nSeahawks fans everywhere.'

In [229]:
from markdown import Markdown
from io import StringIO


def unmark_element(element, stream=None):
    if stream is None:
        stream = StringIO()
    if element.text:
        stream.write(element.text)
    for sub in element:
        unmark_element(sub, stream)
    if element.tail:
        stream.write(element.tail)
    return stream.getvalue()


# patching Markdown
Markdown.output_formats["plain"] = unmark_element
__md = Markdown(output_format="plain")
__md.stripTopLevelTags = False


def unmark(text):
    return __md.convert(text).replace(" \n"," ")

text = unmark(text).replace(" \n"," ")
text

'Dear Falcons fans: are you guys ready for 17 straight weeks of hearing how you fucked up the Super Bowl? I feel your pain. Sincerely, Seahawks fans everywhere.'

In [230]:
import string
text = text.translate(str.maketrans('', '', string.punctuation))
text

'Dear Falcons fans are you guys ready for 17 straight weeks of hearing how you fucked up the Super Bowl I feel your pain Sincerely Seahawks fans everywhere'

In [231]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 202.1 kB/s eta 0:00:0000:0100:01


In [232]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [233]:
def sentiment_scores(sentence):
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
    # polarity_scores method of SentimentIntensityAnalyzer
    # oject gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
    print("Overall sentiment dictionary is : ", sentiment_dict)
    print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
    print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
    print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
    print("Sentence Overall Rated As", end = " ")
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        print("Positive")
    elif sentiment_dict['compound'] <= - 0.05 :
        print("Negative")
    else :
        print("Neutral")

In [234]:
sentiment_scores(text)

Overall sentiment dictionary is :  {'neg': 0.18, 'neu': 0.492, 'pos': 0.328, 'compound': 0.6486}
sentence was rated as  18.0 % Negative
sentence was rated as  49.2 % Neutral
sentence was rated as  32.800000000000004 % Positive
Sentence Overall Rated As Positive


In [235]:
text = reddit_df['Game thread scraped'][0][2][0]
print(text)
text = unmark(text).replace(" \n"," ")
text = text.translate(str.maketrans('', '', string.punctuation))
sentiment_scores(text)

#Fuckin Mark Wahlberg LEFT THE SUPER BOWL EARLY BECAUSE THE PATS WERE DOWN 28-3
Overall sentiment dictionary is :  {'neg': 0.0, 'neu': 0.737, 'pos': 0.263, 'compound': 0.6841}
sentence was rated as  0.0 % Negative
sentence was rated as  73.7 % Neutral
sentence was rated as  26.3 % Positive
Sentence Overall Rated As Positive


In [236]:
import re

def convert_super_bowl(text):
    # Use regular expression to find and replace "super bowl" with "superbowl" (case-insensitive)
    modified_text = re.sub(r'\b(super)\s+(bowl)\b', r'\1\2', text, flags=re.IGNORECASE)
    return modified_text

# Example usage:
original_text = "The Super Bowl is a great event."
converted_text = convert_super_bowl(original_text)
print(converted_text)


The SuperBowl is a great event.


In [289]:
import random

thread_num = []
comment_num = []
comments = []
pos_words = []
neu_words = []
neg_words = []
scores = []
# overall_rating = []

for i in range(10):
    num_1 = random.randrange(len(reddit_df))
    for j in range(50):
        thread_num.append(num_1)
        num_2 = random.randrange(len(reddit_df.iloc[num_1,:].loc['Game thread scraped']))
        if j != 0:
            while num_2 in comment_num[-j:]:
                num_2 = random.randrange(len(reddit_df.iloc[num_1,:].loc['Game thread scraped']))
        comment_num.append(num_2)
        text = reddit_df['Game thread scraped'][num_1][num_2][0]
        # print(text)
        # print("____________________________________________________________________________________________________")
        comments.append(text)
        text = unmark(text).replace(" \n"," ")
        text = convert_super_bowl(text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        from nltk.tokenize import word_tokenize, RegexpTokenizer
        from nltk.sentiment.vader import SentimentIntensityAnalyzer
        tokenized_sentence = nltk.word_tokenize(text)

        sid = SentimentIntensityAnalyzer()
        pos_word_list=[]
        neu_word_list=[]
        neg_word_list=[]

        for word in tokenized_sentence:
            if (sid.polarity_scores(word)['compound']) >= 0.05:
                pos_word_list.append(word)
            elif (sid.polarity_scores(word)['compound']) <= -0.05:
                neg_word_list.append(word)
            else:
                neu_word_list.append(word)                

        # print('Positive:', pos_word_list) 
        pos_words.append(pos_word_list)
        # print('Neutral:', neu_word_list) 
        neu_words.append(neu_word_list)
        # print('Negative:', neg_word_list) 
        neg_words.append(neg_word_list)
        score = sid.polarity_scores(text)
        # print('\nScores:', score,"\n")
        scores.append(score)
        # print("Sentence Overall Rated As", end = " ")
        # if score['compound'] >= 0.05 :
        #     # print("Positive")
        #     overall_rating.append(1)
        # elif score['compound'] <= - 0.05 :
        #     # print("Negative")
        #     overall_rating.append(-1)
        # else :
        #     # print("Neutral")
        #     overall_rating.append(0)

In [366]:
data = {
    "Thread": thread_num,
    "Comment Number": comment_num,
    'Comment Text': comments,
    'Positive Words': pos_words,
    'Neutral Words': neu_words,
    'Negative Words': neg_words,
    'Scores': scores,
    'Overall Rating': overall_rating
}

verifying_df = pd.DataFrame(data)
verifying_df#.to_csv('verifying_scores.csv')

,Thread,Comment Number,Comment Text,Positive Words,Neutral Words,Negative Words,Scores,Overall Rating
0,220,251,He broke crossed the line and had control. How...,[],"[He, crossed, the, line, and, had, control, Ho...","[broke, lose]","{'neg': 0.103, 'neu': 0.813, 'pos': 0.083, 'co...",-1
1,220,396,There were multiple Patriots not lined up prop...,"[play, like]","[There, were, multiple, Patriots, not, lined, ...",[miss],"{'neg': 0.056, 'neu': 0.772, 'pos': 0.172, 'co...",1
2,220,52,THIS WEEK ON WHAT IS A CATCH,[],"[THIS, WEEK, ON, WHAT, IS, A, CATCH]",[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0
3,220,420,8 is neither 5 or 6. Good job D.,[Good],"[8, is, neither, 5, or, 6, job, D]",[],"{'neg': 0.376, 'neu': 0.624, 'pos': 0.0, 'comp...",-1
4,220,283,That's a catch. Don't care what the rules say.,[care],"[Thats, a, catch, Dont, what, the, rules, say]",[],"{'neg': 0.273, 'neu': 0.727, 'pos': 0.0, 'comp...",-1
...,...,...,...,...,...,...,...,...
495,55,189,Wanna know what's gonna happen? Carolina is go...,[win],"[Wan, na, know, whats, gon, na, happen, Caroli...","[fuck, wrong, forgotten]","{'neg': 0.134, 'neu': 0.806, 'pos': 0.06, 'com...",-1
496,55,403,I'm loving these TD celebrations. Glad they're...,"[loving, Glad]","[Im, these, TD, celebrations, theyre, back]",[],"{'neg': 0.0, 'neu': 0.465, 'pos': 0.535, 'comp...",1
497,55,50,Love this hate Patriot fans.,[Love],"[this, Patriot, fans]",[hate],"{'neg': 0.339, 'neu': 0.275, 'pos': 0.385, 'co...",1
498,55,215,STEPHEN GOSTKOWSKI IS STILL PERFECT THIS MONTH!,[PERFECT],"[STEPHEN, GOSTKOWSKI, IS, STILL, THIS, MONTH]",[],"{'neg': 0.0, 'neu': 0.619, 'pos': 0.381, 'comp...",1


In [391]:
df = pd.read_csv('verifying_scores.csv').iloc[:,1:]
def cutoffs(lower_bound, upper_bound, col):
    lst = []
    for i in col:
        if i <= lower_bound:
            lst.append(-1)
        elif i >= upper_bound:
            lst.append(1)
        else:
            lst.append(0)
    return lst
df['Scores'] = df['Scores'].apply(eval)
df['Compound'] = [i['compound'] for i in df['Scores']]
df['Overall Rating'] = cutoffs(-0.05, 0.05, df['Compound'])
df['Accurate'] = (df['My rating'] == df['Overall Rating']).astype(int)
df.loc[(df['Overall Rating'] == 0) & (df['My rating'] != 0), 'Accurate'] = \
    df.loc[(df['Overall Rating'] == 0) & (df['My rating'] != 0), 'My rating']/2
df.loc[df['My rating'] == -99, 'Accurate'] = -99
df.loc[df['My rating'].isna(), 'Accurate'] = np.NaN
print(df.loc[:,'Accurate'].value_counts(normalize=True))
df

 1.0     0.521053
 0.0     0.247368
-0.5     0.110526
-99.0    0.094737
 0.5     0.026316
Name: Accurate, dtype: float64


,Thread,Comment Number,Comment Text,Positive Words,Neutral Words,Negative Words,Scores,My rating,Compound,Overall Rating,Accurate
0,220,251,He broke crossed the line and had control. How...,[],"['He', 'crossed', 'the', 'line', 'and', 'had',...","['broke', 'lose']","{'neg': 0.103, 'neu': 0.813, 'pos': 0.083, 'co...",-1.0,-0.1386,-1,1.0
1,220,396,There were multiple Patriots not lined up prop...,"['play', 'like']","['There', 'were', 'multiple', 'Patriots', 'not...",['miss'],"{'neg': 0.056, 'neu': 0.772, 'pos': 0.172, 'co...",-1.0,0.5106,1,0.0
2,220,52,THIS WEEK ON WHAT IS A CATCH,[],"['THIS', 'WEEK', 'ON', 'WHAT', 'IS', 'A', 'CAT...",[],"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,0.0000,0,1.0
3,220,420,8 is neither 5 or 6. Good job D.,['Good'],"['8', 'is', 'neither', '5', 'or', '6', 'job', ...",[],"{'neg': 0.376, 'neu': 0.624, 'pos': 0.0, 'comp...",-99.0,-0.3412,-1,-99.0
4,220,283,That's a catch. Don't care what the rules say.,['care'],"['Thats', 'a', 'catch', 'Dont', 'what', 'the',...",[],"{'neg': 0.273, 'neu': 0.727, 'pos': 0.0, 'comp...",1.0,-0.3875,-1,0.0
...,...,...,...,...,...,...,...,...,...,...,...
485,55,189,Wanna know what's gonna happen? Carolina is go...,['win'],"['Wan', 'na', 'know', 'whats', 'gon', 'na', 'h...","['fuck', 'wrong', 'forgotten']","{'neg': 0.134, 'neu': 0.806, 'pos': 0.06, 'com...",NaN,-0.5719,-1,NaN
486,55,403,I'm loving these TD celebrations. Glad they're...,"['loving', 'Glad']","['Im', 'these', 'TD', 'celebrations', 'theyre'...",[],"{'neg': 0.0, 'neu': 0.465, 'pos': 0.535, 'comp...",NaN,0.7845,1,NaN
487,55,50,Love this hate Patriot fans.,['Love'],"['this', 'Patriot', 'fans']",['hate'],"{'neg': 0.339, 'neu': 0.275, 'pos': 0.385, 'co...",NaN,0.1280,1,NaN
488,55,215,STEPHEN GOSTKOWSKI IS STILL PERFECT THIS MONTH!,['PERFECT'],"['STEPHEN', 'GOSTKOWSKI', 'IS', 'STILL', 'THIS...",[],"{'neg': 0.0, 'neu': 0.619, 'pos': 0.381, 'comp...",NaN,0.5719,1,NaN


In [393]:
for idx in df['Thread'].unique()[:4]:
    sub_df = df[df['Thread'] == idx]
    print(f"Index: {idx}")
    print(f"Length: {len(sub_df)}")
    print(sub_df.loc[:,'Accurate'].value_counts(normalize=True))

Index: 220
Length: 46
 1.0     0.478261
 0.0     0.239130
-99.0    0.173913
-0.5     0.108696
Name: Accurate, dtype: float64
Index: 424
Length: 48
 1.0     0.500000
 0.0     0.229167
-99.0    0.145833
-0.5     0.125000
Name: Accurate, dtype: float64
Index: 603
Length: 47
 1.0    0.574468
 0.0    0.319149
-0.5    0.085106
 0.5    0.021277
Name: Accurate, dtype: float64
Index: 416
Length: 49
 1.0     0.530612
 0.0     0.204082
-0.5     0.122449
 0.5     0.081633
-99.0    0.061224
Name: Accurate, dtype: float64


## Generating proportions

In [421]:
reddit_df.head(2)

,Season,Week,Day,Date,Time (ET),Unnamed: 5,Favorite,Score,Spread,Unnamed: 9,...,GameHub/RedZone,Game thread link?,Post-game thread link?,Game thread,Post game thread,GameHub Scraped,Game thread scraped,Post game thread Scraped,Subscribers (in thousands),all comments
0,2017,1,Thu,2017-09-07,8:30,@,New England Patriots,L 27-42,L -8,NaN,...,NaN,NaN,NaN,https://www.reddit.com/r/nfl/comments/6yr619/g...,https://www.reddit.com/r/nfl/comments/6ysc10/p...,NaN,"[3193, (Dear Falcons fans: are you guys ready ...","[2781, (I'm imagining a world where the Patrio...",640,"[3193, (Dear Falcons fans: are you guys ready ..."
1,2017,1,Sun,2017-09-10,1:00,@,Buffalo Bills,W 21-12,W -7,NaN,...,https://www.reddit.com/r/nfl/comments/6z90qu/g...,NaN,1.0,https://www.reddit.com/r/nfl/comments/6z97il/g...,https://www.reddit.com/r/nfl/comments/6zajs1/p...,"[240, (Bitter-sweet start to the NFL season wi...","[109, (I'm a Jets fan. I'm a Texas A&M fan. Ye...","[120, (Bills number 1 seed in the AFCE!, 194),...",640,"[109, (I'm a Jets fan. I'm a Texas A&M fan. Ye..."


In [493]:
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def text_analyser(text):
    sid_obj = SentimentIntensityAnalyzer()
    text = unmark(text).replace(" \n"," ")
    text = convert_super_bowl(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    sentiment_dict = sid_obj.polarity_scores(text)
    if sentiment_dict['compound'] >= 0.05 :
        return 1
    elif sentiment_dict['compound'] <= - 0.05 :
        return -1
    else :
        return 0

def list_analyser(lst):
    lst_analysed = [text_analyser(i[0]) for i in lst]
    return lst_analysed
    
def prop_calculator(analysed_lst):
    proportion_pos = analysed_lst.count(1) / len(analysed_lst)
    proportion_neg = analysed_lst.count(-1) / len(analysed_lst)
    return (proportion_pos, proportion_neg)

alpha = list_analyser(reddit_df['all comments'][0])
alpha[:5]


TypeError: 'int' object is not subscriptable

In [490]:
lst = reddit_df['all comments'][0]
lst

'[3193, (\'Dear Falcons fans: are you guys ready for 17 straight weeks of hearing how you fucked up the Super Bowl? I feel your pain. \\n\\nSincerely, \\n\\n\\nSeahawks fans everywhere.\', 2993), (\'#Fuckin Mark Wahlberg LEFT THE SUPER BOWL EARLY BECAUSE THE PATS WERE DOWN 28-3\', 1543), ("I\'m naked and ready to go boys", 1138), ("Fuck this what the fuck we aren\'t even playing", 1107), (\'There are actual people. Actual human beings. Who sat in a room one day and said "hey, let\\\'s play \\\'Jump Around\\\' when they unveil the Super Bowl thing."\', 1075), (\'WE MADE IT! THE OFF-SEASON IS OVER AND I LOVE ALL OF YOU. \', 1060), (\'"Sometimes I think the Patriots have better Buffalo players than Buffalo"\\n\\nFuck off.\', 832), ("Roses are red, \\n\\nViolets are blue.\\n\\nThis doesn\'t rhyme,\\n\\nFuck Goodell. ", 758), (\'Where do I apply for the "Guy who holds the Microsoft Surface™ replay tablet" position?  \', 663), (\'Game postponed for rain due to the insurmountable amount of cu

In [ ]:
reddit_df['sentiment_scores'] = reddit_df['all comments'].apply(list_analyser)
reddit_df['props'] = reddit_df['sentiment_scores'].apply(prop_calculator)
reddit_df.head()

In [238]:
import nltk
import nltk.corpus
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/rohanjha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [263]:
token = word_tokenize(text)
token

['Bosa',
 'literally',
 'gets',
 'held',
 'next',
 'to',
 'that',
 'like',
 'come',
 'on']

In [264]:
from nltk.stem import PorterStemmer
pst = PorterStemmer()
stemmed_words = [pst.stem(x) for x in token]
stemmed_words

['bosa', 'liter', 'get', 'held', 'next', 'to', 'that', 'like', 'come', 'on']

In [265]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(x) for x in stemmed_words]
lemmatized_words

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rohanjha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['bosa', 'liter', 'get', 'held', 'next', 'to', 'that', 'like', 'come', 'on']

In [266]:
from nltk.corpus import stopwords
a = set(stopwords.words('english'))
no_stopwords = [x for x in lemmatized_words if x not in a]
no_stopwords

['bosa', 'liter', 'get', 'held', 'next', 'like', 'come']

In [267]:
nltk.download('averaged_perceptron_tagger')
for word in token:
    print(nltk.pos_tag([word]))

[('Bosa', 'NNP')]
[('literally', 'RB')]
[('gets', 'VBZ')]
[('held', 'NN')]
[('next', 'JJ')]
[('to', 'TO')]
[('that', 'IN')]
[('like', 'IN')]
[('come', 'VB')]
[('on', 'IN')]


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rohanjha/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [268]:
from nltk.probability import FreqDist
fdist = FreqDist([pst.stem(x) for x in no_stopwords])
fdist

FreqDist({'bosa': 1, 'liter': 1, 'get': 1, 'held': 1, 'next': 1, 'like': 1, 'come': 1})